We now have our ready encoder architecture, now we need to make the decoder which takes input of encoders and then preidcting out the predictions for our reactions which will predict the reagent, solvent and catalyst in this case

# Importing Libraries

In [17]:
!pip install torch-geometric rdkit-pypi fasttext

In [18]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

from nltk.tokenize import word_tokenize
from collections import Counter
import nltk
import spacy
import fasttext

import torch
import torch.nn as nn
import torchtext
import torch.nn.functional as F
from torch.utils.data import Dataset,dataloader
from torch.nn.utils.rnn import pack_padded_sequence,pad_packed_sequence,pad_sequence

from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.data import DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [19]:
from rdkit import RDLogger

Chem.MolFromSmiles('c1cncc1')
RDLogger.DisableLog('rdApp.*')
Chem.MolFromSmiles('c1cncc1')

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Importing Dataset

In [22]:
df=pd.read_csv('/content/drive/MyDrive/Chiros Dataset/choriso_public.tsv',sep='\t')
df.head()

,canonic_rxn,rxnmapper_aam,reagent,solvent,catalyst,yield
0,CCO.O=S1(=O)C=Cc2ccccc21.[H][H].[Pd]>>O=S1(=O)...,CCO.[O:1]=[S:2]1(=[O:3])[CH:4]=[CH:5][c:6]2[cH...,hydrogen,ethanol,palladium on activated charcoal,100.0
1,O=S1(=O)C=Cc2ccccc21.[Na+].[OH-].[Zn]>>O=S1(=O...,[O:1]=[S:2]1(=[O:3])[CH:4]=[CH:5][c:6]2[cH:7][...,sodium hydroxide|zinc,empty,empty,0.0
2,CCO.O=S1(=O)C=Cc2ccccc21.[Pd]>>O=S1(=O)CCc2ccc...,CCO.[O:1]=[S:2]1(=[O:3])[CH:4]=[CH:5][c:6]2[cH...,palladium on activated charcoal|ethanol,empty,empty,0.0
3,CO.O=C1CCCN1C1CCN(Cc2ccccc2)CC1.O=C[O-].[NH4+]...,CO.[O:1]=[C:2]1[CH2:3][CH2:4][CH2:5][N:6]1[CH:...,palladium 10 on activated carbon|ammonium formate,methanol,empty,100.0
4,CC(C)(C)OC(=O)N1CC2CC(CN(Cc3ccccc3)C2)C1.CCO.[...,[CH3:1][C:2]([CH3:3])([CH3:4])[O:5][C:6](=[O:7...,hydrogen,ethanol,palladium on activated charcoal,51.0


# Features File

In [23]:
def featurize_molecule(mol):
    # Compute Morgan fingerprints for each atom
    atom_features = []
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        atom_feature = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, atomIndices=[idx])
        atom_features.append(np.array(atom_feature))

    return np.array(atom_features)

In [24]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)

    # Add explicit hydrogens
    mol = Chem.AddHs(mol)

    # Generate 3D coordinates for visualization
    AllChem.EmbedMolecule(mol, randomSeed=42)  # You can choose any seed value

    # Get atom features and adjacency matrix
    num_atoms = mol.GetNumAtoms()
    atom_features = np.zeros((num_atoms, 3))  # You may need to adjust the feature dimensions
    adjacency_matrix = np.zeros((num_atoms, num_atoms))

    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        adjacency_matrix[i, j] = adjacency_matrix[j, i] = 1  # Adjacency matrix is symmetric

    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        atom_features[idx, 0] = atom.GetAtomicNum()  # Atom type or atomic number
        atom_features[idx, 1] = atom.GetTotalNumHs()  # Number of hydrogen atoms
        atom_features[idx, 2] = atom.GetFormalCharge()  # Formal charge

    # Convert to PyTorch tensors
    atom_features = torch.tensor(atom_features, dtype=torch.float)

    # Create edge_index using the adjacency matrix
    edge_index = torch.tensor(np.column_stack(np.where(adjacency_matrix)), dtype=torch.long)

    # Create PyTorch Geometric data object
    data = Data(x=atom_features, edge_index=edge_index.t().contiguous())  # Transpose edge_index

    return data

# GCN

In [51]:
class PositionalEncoding(nn.Module):
    def __init__(self, input_size, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.encoding = nn.Embedding(max_len, input_size)

    def forward(self, x):
        x = x.to(device)
        positions = torch.arange(0, x.size(1), device=device).unsqueeze(0)
        positions = positions.expand(x.size(0), -1)  # Expand along the batch dimension
        positions = positions.to(device)
        return x + self.encoding(positions)

In [62]:
class DistanceAttentionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(DistanceAttentionEncoder, self).__init__()

        self.embedding = PositionalEncoding(input_size)
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)

    def pairwise_distances(self, x):
        distances = torch.norm(x[:, None, :] - x, dim=-1, p=2)
        return distances

    def forward(self, input_sequence):
        embedded_sequence = self.embedding(input_sequence)
        encoded_sequence = self.encoder(embedded_sequence)
        attention_scores = self.decoder(torch.tanh(encoded_sequence))
        attention_weights = self.softmax(attention_scores)
        context_vector = torch.sum(encoded_sequence * attention_weights, dim=1)

        return context_vector

In [63]:
class MultiHeadAttention(nn.Module):
    def __init__(self, in_channels, out_channels, attention_heads=1):
        super().__init__()
        self.W_q = nn.Linear(in_channels, out_channels)
        self.W_k = nn.Linear(in_channels, out_channels)
        self.W_v = nn.Linear(in_channels, out_channels)
        self.attention_heads = attention_heads

    def forward(self, x):
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)

        q = q.view(-1, self.attention_heads, q.size(-1))
        k = k.view(-1, self.attention_heads, k.size(-1))
        v = v.view(-1, self.attention_heads, v.size(-1))

        scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(q.size(-1), dtype=torch.float32))
        attention_weights = F.softmax(scores, dim=-1)
        attention_output = torch.matmul(attention_weights, v).view(x.size(0), -1)

        return attention_output


In [64]:
class GATModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, external_attention_heads=None):
        super(GATModel, self).__init__()

        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.external_attention = MultiHeadAttention(hidden_channels * heads, hidden_channels, attention_heads=external_attention_heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)
        self.external_attention_heads = external_attention_heads

    def forward(self, data):
        data = data.to(device)
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        if self.external_attention_heads is not None:
            external_attention_output = self.external_attention(x)
            x = torch.cat([x, external_attention_output], dim=-1)

        x = self.conv2(x, edge_index)

        self.distance_attention_encoder = DistanceAttentionEncoder(x.size(1), hidden_size=64).to(device)
        distance_attention_output = self.distance_attention_encoder(x.unsqueeze(0))

        return x


# Seperating compunds in the SMILES

In [55]:
def separate_compounds(smiles_reaction):
    # Split the reaction string using '>>' as the separator
    compounds = smiles_reaction.split(">>")

    # Ensure that there are exactly two compounds
    if len(compounds) == 2:
        reactant = compounds[0].strip()
        product = compounds[1].strip()
        return reactant, product
    else:
        raise ValueError("Invalid SMILES reaction format. Expected one '>>' separator.")

# Given SMILES reaction
smiles_reaction = "O=C1CCCN1C1CCN(Cc2ccccc2)CC1>>O=C1CCCN1C1CCNCC1"

# Separate compounds
reactant, product = separate_compounds(smiles_reaction)

# Print the separated compounds
print("Reactant:", reactant)
print("Product:", product)


Reactant: O=C1CCCN1C1CCN(Cc2ccccc2)CC1
Product: O=C1CCCN1C1CCNCC1


# Cross Attention for both
Using cross attention to concatenate for both the compounds into a single embedding space

In [65]:
def concatenate_with_cross_attention(emb1, emb2, out_channels, heads):
    if emb1.shape[0] > emb2.shape[0]:
        linear_projection = nn.Linear(emb2.shape[0], emb1.shape[0]).to(device)
        emb2 = linear_projection(emb2.T).T
    elif emb1.shape[0] < emb2.shape[0]:
        linear_projection = nn.Linear(emb1.shape[0], emb2.shape[0]).to(device)
        emb1 = linear_projection(emb1.T).T

    concatenated_emb = torch.cat((emb1, emb2), dim=1)

    multihead_attention = nn.MultiheadAttention(embed_dim=2 * out_channels, num_heads=heads).to(device)
    cross_attended_emb, _ = multihead_attention(concatenated_emb, concatenated_emb, concatenated_emb)

    return cross_attended_emb

In [66]:
def get_cross_attention_output(smiles_string, train_mode=True):
    reactant, product = separate_compounds(smiles_string)
    graph_data_reactant = smiles_to_graph(reactant)
    graph_data_product = smiles_to_graph(product)
    graph_data_reactant = graph_data_reactant.to(device)
    graph_data_product = graph_data_product.to(device)

    in_channels = graph_data_reactant.x.size(1)
    hidden_channels = 64
    out_channels = 32
    heads = 2
    gat_model_reactant = GATModel(in_channels, hidden_channels, out_channels, heads).to(device)

    if train_mode:
        gat_model_reactant.train()
    else:
        gat_model_reactant.eval()

    output_reactant = gat_model_reactant(graph_data_reactant)

    in_channels = graph_data_product.x.size(1)
    hidden_channels = 64
    out_channels = 32
    heads = 2
    gat_model_product = GATModel(in_channels, hidden_channels, out_channels, heads).to(device)

    if train_mode:
        gat_model_product.train()
    else:
        gat_model_product.eval()

    output_product = gat_model_product(graph_data_product)

    out = concatenate_with_cross_attention(output_reactant, output_product, out_channels=32, heads=2)

    return out

In [67]:
smiles_string = "CC(C)(C)OC(=O)N1CC2CC(CN(Cc3ccccc3)C2)C1>>CC(C)(C)OC(=O)N1CC2CNCC(C2)C1"
out=get_cross_attention_output(smiles_string)

# Vocab Size and Padding

In [68]:
def pad_sequence(sequence, max_length, padding_token='<PAD>'):
    return sequence + [padding_token] * (max_length - len(sequence))

In [69]:
df=df.iloc[:10000,:]

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   canonic_rxn    100 non-null    object 
 1   rxnmapper_aam  100 non-null    object 
 2   reagent        100 non-null    object 
 3   solvent        100 non-null    object 
 4   catalyst       100 non-null    object 
 5   yield          100 non-null    float64
dtypes: float64(1), object(5)
memory usage: 4.8+ KB


In [71]:
columns_to_process = ['reagent', 'solvent', 'catalyst']

for column in columns_to_process:
    # Tokenize and build vocabularies for each column
    tokens = [token for item in df[column] for token in word_tokenize(str(item))]
    vocab = set(tokens)
    vocab_size = len(vocab)

    # Pad sequences to a common size
    max_seq_length = max(len(token) for token in tokens)

    # Pad sequences in the DataFrame
    df[column] = df[column].apply(lambda x: pad_sequence(word_tokenize(str(x)), max_seq_length))

    # Print results
    print(f"{column.capitalize()} Vocabulary Size: {vocab_size}")
    print(f"Padded {column.capitalize()}:\n{df[column]}\n")

Reagent Vocabulary Size: 149
Padded Reagent:
0     [hydrogen, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, ...
1     [sodium, hydroxide|zinc, <PAD>, <PAD>, <PAD>, ...
2     [palladium, on, activated, charcoal|ethanol, <...
3     [palladium, 10, on, activated, carbon|ammonium...
4     [hydrogen, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, ...
                            ...                        
95    [tetrachloromethane|bromine, <PAD>, <PAD>, <PA...
96    [thionyl, chloride, <PAD>, <PAD>, <PAD>, <PAD>...
97    [thionyl, chloride, <PAD>, <PAD>, <PAD>, <PAD>...
98    [oxalyl, dichloride|n, ,, n-dimethyl-formamide...
99    [thionyl, chloride, <PAD>, <PAD>, <PAD>, <PAD>...
Name: reagent, Length: 100, dtype: object

Solvent Vocabulary Size: 34
Padded Solvent:
0     [ethanol, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <...
1     [empty, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PA...
2     [empty, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PA...
3     [methanol, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, ...
4     [ethanol, <PAD>, <PAD>

<ipython-input-71-a37ee099bce5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: pad_sequence(word_tokenize(str(x)), max_seq_length))


# A full Seq2Seq Network

In [80]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size_condition1, vocab_size_condition2, vocab_size_condition3, d_model=512, nhead=8, num_layers=6, enc_dim=64, max_seq=100):
        super(TransformerDecoder, self).__init__()

        self.vocab_size_condition1 = vocab_size_condition1
        self.vocab_size_condition2 = vocab_size_condition2
        self.vocab_size_condition3 = vocab_size_condition3
        self.d_model = d_model
        self.linear_layer = nn.Linear(vocab_size_condition1, d_model)

        self.embedding1 = nn.Embedding(vocab_size_condition1, d_model)
        self.embedding2 = nn.Embedding(vocab_size_condition2, d_model)
        self.embedding3 = nn.Embedding(vocab_size_condition3, d_model)

        self.transformer_layers = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model, nhead),
            num_layers
        )

        self.fc_condition1 = nn.Linear(d_model, vocab_size_condition1)
        self.fc_condition2 = nn.Linear(d_model, vocab_size_condition2)
        self.fc_condition3 = nn.Linear(d_model, vocab_size_condition3)

        self.softmax_condition1 = nn.Softmax(dim=1)
        self.softmax_condition2 = nn.Softmax(dim=1)
        self.softmax_condition3 = nn.Softmax(dim=1)

        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead)

        self.linear_proj = nn.Linear(max_seq, enc_dim)
        self.max_seq = max_seq

        self.final_linear_layer1 = nn.Linear(d_model, max_seq)
        self.final_linear_layer2 = nn.Linear(d_model, max_seq)
        self.final_linear_layer3 = nn.Linear(d_model, max_seq)

    def forward(self, encoded_input, target_sequence):
        self.emb_cond1 = self.embedding1(target_sequence[0].to(device))
        self.emb_cond2 = self.embedding2(target_sequence[1].to(device))
        self.emb_cond3 = self.embedding3(target_sequence[2].to(device))

        self.comb_emb = self.emb_cond1 + self.emb_cond2 + self.emb_cond3
        self.comb_emb = self.comb_emb.squeeze(0)

        memory = torch.rand(1, self.max_seq, 512).to(device)
        trans_decoder = self.transformer_layers(self.comb_emb, memory=memory)
        new_size = (trans_decoder.size(0) * trans_decoder.size(1), -1)
        trans_decoder_2d = trans_decoder.view(*new_size)

        proj_trans_decoder = self.linear_proj(trans_decoder_2d.T).T

        tensor1 = proj_trans_decoder.unsqueeze(1)
        tensor1 = tensor1.permute(1, 2, 0)
        tensor2 = encoded_input.unsqueeze(0)

        attention_weights = F.softmax(torch.bmm(tensor1, tensor2.permute(0, 2, 1)), dim=-1)
        cross_attention_result = torch.bmm(attention_weights, tensor2)
        cross_attention_result = cross_attention_result.squeeze(1)
        new_size = (cross_attention_result.size(0) * cross_attention_result.size(1), -1)
        cross_attention_result = cross_attention_result.view(*new_size)

        final_output1 = self.final_linear_layer1(cross_attention_result.T)
        final_output_summed1 = final_output1.sum(dim=0)

        final_output2 = self.final_linear_layer2(cross_attention_result.T)
        final_output_summed2 = final_output2.sum(dim=0)

        final_output3 = self.final_linear_layer3(cross_attention_result.T)
        final_output_summed3 = final_output3.sum(dim=0)

        return F.relu(final_output_summed1), F.relu(final_output_summed2), F.relu(final_output_summed3)


In [81]:
class TransformerSeq2Seq(nn.Module):
    def __init__(self, vocab_size_condition1, vocab_size_condition2, vocab_size_condition3, decoder_d_model=512, decoder_nhead=8, decoder_layers=6, enc_dim=64):
        super(TransformerSeq2Seq, self).__init__()

        self.decoder = TransformerDecoder(vocab_size_condition1, vocab_size_condition2, vocab_size_condition3,
                                         d_model=decoder_d_model, nhead=decoder_nhead, num_layers=decoder_layers, enc_dim=enc_dim)

    def forward(self, input_sequence, targets, train_mode=True):
        self.hidden_state = get_cross_attention_output((input_sequence), train_mode)

        self.decoder = self.decoder.to(device)

        if train_mode:
            self.decoder.train()
        else:
            self.decoder.eval()

        output_probs_condition1, output_probs_condition2, output_probs_condition3 = self.decoder(self.hidden_state, targets)

        return output_probs_condition1,output_probs_condition2,output_probs_condition3

# Conversion Into Embeddings
As we knwo have our tokeinzed data, we need to first convert them into their respective embeddings, to show the working for we have used the token to int mapping which randomly assigns them integers and add +1 to each of the new word

In [126]:
# Dictionary to store mappings of tokens to integers
token_to_int_mapping = {}

def token_to_int(token):
    global token_to_int_mapping

    if token not in token_to_int_mapping:
        # Assign a random number to the new token
        random_int = len(token_to_int_mapping) + 1  # Start from 1 to avoid conflict with 0 for padding
        token_to_int_mapping[token] = random_int

    return token_to_int_mapping[token]

def tokenize_and_pad_sequence(sequence, max_seq_length):
    # Convert <PAD> values to 0
    tokenized_sequence = [0 if token == '<PAD>' else token_to_int(token) for token in sequence]

    # Pad the sequence with zeros if it's shorter than max_seq_length
    padded_sequence = tokenized_sequence + [0] * (max_seq_length - len(tokenized_sequence))

    # Convert the sequence to a PyTorch tensor
    tensor_sequence = torch.tensor(padded_sequence, dtype=torch.long)

    return tensor_sequence

In [127]:
max_seq_length = 100  # Adjust this according to your requirements
tokenized_cond1 = tokenize_and_pad_sequence(df['reagent'][1], max_seq_length)
tensor_cond1 = tokenized_cond1.unsqueeze(0)  # Assuming batch size is 1

# Example usage for the second condition (solvent)
tokenized_cond2 = tokenize_and_pad_sequence(df['solvent'][2], max_seq_length)
tensor_cond2 = tokenized_cond2.unsqueeze(0)  # Assuming batch size is 1

# Example usage for the third condition (catalyst)
tokenized_cond3 = tokenize_and_pad_sequence(df['catalyst'][10], max_seq_length)
tensor_cond3 = tokenized_cond3.unsqueeze(0)  # Assuming batch size is 1

In [128]:
tensor_cond1.shape

torch.Size([1, 100])

# Making DataLoader

In [129]:
class ReactionDataset(Dataset):
    def __init__(self, dataframe, max_seq_length):
        self.dataframe = dataframe
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        reagent_sequence = tokenize_and_pad_sequence(df['reagent'][idx], max_seq_length)
        tensor_reagent = torch.tensor(reagent_sequence).unsqueeze(0)

        solvent_sequence = tokenize_and_pad_sequence(df['solvent'][idx], max_seq_length)
        tensor_solvent = torch.tensor(solvent_sequence).unsqueeze(0)

        catalyst_sequence = tokenize_and_pad_sequence(df['catalyst'][idx], max_seq_length)
        tensor_catalyst = torch.tensor(catalyst_sequence).unsqueeze(0)


        canonic_rxn = self.dataframe['canonic_rxn'][idx]

        return canonic_rxn,[tensor_reagent,tensor_solvent,tensor_catalyst]

In [130]:
reaction_dataset = ReactionDataset(df, max_seq_length=100)
reaction_dataloader = DataLoader(reaction_dataset, batch_size=1, shuffle=True)

# Training Loop

In [131]:
def train_model(model, dataloader, optimizer, num_epochs=10):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, targets in reaction_dataloader:
            inputs = inputs[0]
            optimizer.zero_grad()

            outputs1, outputs2, outputs3 = model(inputs, targets)
            outputs1 = outputs1.unsqueeze(0)
            outputs2 = outputs2.unsqueeze(0)
            outputs3 = outputs3.unsqueeze(0)

            outputs1 = outputs1.float()
            outputs2 = outputs2.float()
            outputs3 = outputs3.float()

            targets1 = targets[0].squeeze(0).float().to(device)
            targets2 = targets[1].squeeze(0).float().to(device)
            targets3 = targets[2].squeeze(0).float().to(device)

            loss1 = F.mse_loss(outputs1, targets1)
            loss2 = F.mse_loss(outputs2, targets2)
            loss3 = F.mse_loss(outputs3, targets3)

            epoch_loss = loss1 + loss2 + loss3

            epoch_loss.backward()
            optimizer.step()

            total_loss += epoch_loss

        average_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}")

    return average_loss

# Assuming your model and dataloader are defined somewhere
# Create an instance of the model
model = TransformerSeq2Seq(vocab_size_condition1=1000, vocab_size_condition2=1000, vocab_size_condition3=1000)

# Assuming you have a DataLoader named reaction_dataloader
# Initialize the loss function and optimizer

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
loss=train_model(model, reaction_dataloader, optimizer, num_epochs=5)


<ipython-input-129-ae623d6fc907>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_reagent = torch.tensor(reagent_sequence).unsqueeze(0)
<ipython-input-129-ae623d6fc907>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_solvent = torch.tensor(solvent_sequence).unsqueeze(0)
<ipython-input-129-ae623d6fc907>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_catalyst = torch.tensor(catalyst_sequence).unsqueeze(0)


Epoch 1/5, Loss: 301.7540
Epoch 2/5, Loss: 296.9651
Epoch 3/5, Loss: 293.9265
Epoch 4/5, Loss: 316.3997
Epoch 5/5, Loss: 295.0976


# Inferencing

In [176]:
def find_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key

    return None

In [183]:
def decode_input(encoded_input, token_to_int_mapping):
    decoded_input=[]
    for idx in encoded_input:
      if int(idx) !=0 and find_key_by_value(token_to_int_mapping, int(idx)) is not None:
        decoded_input.append(find_key_by_value(token_to_int_mapping, int(idx)))
    return decoded_input


In [190]:
def inference(model,input_encoding):
    # Load the trained model
    # model = TransformerSeq2Seq(vocab_size_condition1=1000, vocab_size_condition2=1000, vocab_size_condition3=1000)
    # model_state_dict = torch.load('your_model_checkpoint.pth')
    # model.load_state_dict(model_state_dict)
    model = model.to(device)
    model.eval()


    # Create a placeholder tensor for target_sequence during inference
    placeholder_target = [torch.zeros((1,1,100)).long().to(device),torch.zeros((1,1,100)).long().to(device),torch.zeros((1,1,100)).long().to(device)]  # Adjust max_sequence_length accordingly7



    # Perform inference
    with torch.no_grad():
        # Pass only input_encoding and the placeholder_target to the model during inference
        output_probs_condition1, output_probs_condition2, output_probs_condition3 = model(input_encoding, targets=placeholder_target, train_mode=False)

    # Post-process output if necessary
    output_condition1 = output_probs_condition1.cpu().numpy()
    output_condition2 = output_probs_condition2.cpu().numpy()
    output_condition3 = output_probs_condition3.cpu().numpy()

    decoded_condition1 = decode_input(output_probs_condition1, token_to_int_mapping)
    decoded_condition2 = decode_input(output_probs_condition2, token_to_int_mapping)
    decoded_condition3 = decode_input(output_probs_condition3, token_to_int_mapping)

    reactant,product= separate_compounds(input_encoding)

    print("This is the original Reaction", input_encoding,'The reactant is ',reactant, 'The product is ',product)
    print("The Reagant of the original Reaction is",' '.join(decoded_condition1))
    print("The Solvent of the original Reaction is",' '.join(decoded_condition2))
    print("The Catalyst of the original Reaction is",' '.join(decoded_condition3))


    return output_condition1, output_condition2, output_condition3

In [191]:
o1,o2,o3=inference(model,'CC(C)(C)OC(=O)N1CC2CC(CN(Cc3ccccc3)C2)C1>>CC(C)(C)OC(=O)N1CC2CNCC(C2)C1')

This is the original Reaction CC(C)(C)OC(=O)N1CC2CC(CN(Cc3ccccc3)C2)C1>>CC(C)(C)OC(=O)N1CC2CNCC(C2)C1 The reactant is  CC(C)(C)OC(=O)N1CC2CC(CN(Cc3ccccc3)C2)C1 The product is  CC(C)(C)OC(=O)N1CC2CNCC(C2)C1
The Reagant of the original Reaction is tris sodium
The Solvent of the original Reaction is 
The Catalyst of the original Reaction is empty empty empty empty sodium


In [188]:
print(o1)

[31.234661   5.64433    6.460329   0.        15.717079   3.0150023
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.       